In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)

from analysis.lib.pq import pq_tools;reload(pq_tools)

%matplotlib inline
from analysis.lib.fitting import fit, common

#folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data\Purify_25percent_theta'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data\Purify_XX_First_Attempts'

filename_contains = 'Purify_'

from analysis.lib.purification import purify_TPQI_from_Puri_Data as TPQI; reload(TPQI)

In [ ]:
### find timestamps for all three experiments (for pi/6)
days_25 = ['20160720','20160721','20160722','20160724','20160725','20160807','20160808']
days_First = ['20160714','20160715','20160716','20160718','20160719','20160726','20160727']
days_First_new_timings = ['20160816']
days_25_new_timings = ['20160811']

all_lt4 = []
offsets = [] # Hold offset to compensate for timing change for new APD
correction_time = 2791.5e3-2772.5e3
offsets_ch1 = []
start_offset_ch1 = -1.8e3

days = days_First

for d in days:

    tstamp_lt4 = TPQI.find_tstamps_of_day([],d,contains=filename_contains,analysis_folder = folder_lt4)
    all_lt4.extend(tstamp_lt4)
    offsets.extend(np.zeros(np.shape(tstamp_lt4)))
    offsets_ch1.extend(np.zeros(np.shape(tstamp_lt4)))
days = days_First_new_timings
for d in days:

    tstamp_lt4 = TPQI.find_tstamps_of_day([],d,contains=filename_contains,analysis_folder = folder_lt4)

    all_lt4.extend(tstamp_lt4)
    offsets.extend(np.zeros(np.shape(tstamp_lt4))+correction_time)
    offsets_ch1.extend(np.zeros(np.shape(tstamp_lt4))+start_offset_ch1)
print 'No of files: ', len(all_lt4)

In [ ]:
### define parameters
start_ch0 = 2772.5e3

window_length = 40e3
pulse_sep = 0
number_of_pulses = 1

In [ ]:
### get coincidences
# execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
reload(TPQI)
reload(pq_tools)
dts,dts_offset, filtered_dts,filtered_dts_offset = TPQI.TPQI_analysis(folder_lt4,all_lt4,offsets,offsets_ch1,start_ch0,start_ch0,
                                       window_length,
                                       pulse_sep,
                                       number_of_pulses, Verbose = False)

len(dts),len(dts_offset),len(filtered_dts),len(filtered_dts_offset)

In [ ]:
### plot result

lim = 50

hist_binsize = 4
bins = np.arange(min(dts)-100.5,max(dts)+100.5,hist_binsize)


fig = plt.figure()
ax = fig.add_subplot(111)

y,x=np.histogram(filtered_dts, bins=bins)
bin_centres = x[:-1] + (x[1] - x[0])/2
ax.plot(bin_centres,y)

y,x=np.histogram(filtered_dts_offset, bins=bins)
bin_centres = x[:-1] + (x[1] - x[0])/2
ax.plot(bin_centres,y)


ax.set_xlabel('Difference time (ns)')
ax.set_ylabel('Coincidences in bin')
# ax.set_range
ax.set_title('TPQI')
ax.set_xlim([-lim,lim])
# ax.set_ylim([0,30])

plt.show()
plt.close('all')




In [ ]:
counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))
counts_in_next_pulse = np.sum((filtered_dts_offset<200) & (filtered_dts_offset>-200))

print 'coincidence with next pulse',counts_in_next_pulse 
print 'coincidences in same pulse',counts_in_same_pulse

tail_lt3 = 7.2 #9.1975 for SIL 3
tail_lt4 = 5.1514

p1 = tail_lt3
p2 = tail_lt4

Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
# Visibility = 0.5
print 'assumed tail LT3/LT4',tail_lt3,tail_lt4
print 'distiungishable expected vis', Visibility
print 'measured Visibility', counts_in_same_pulse/float(counts_in_next_pulse)
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility


In [ ]:
def sweep_tpqi_param(param_name,sweep_pts):
    vis = []
    vis_u = []
    counts_filt = []
    counts_filt_next = []
    
    initialVal = tpqi_dict[param_name]
    for p in sweep_pts:
        tpqi_dict[param_name] = p
        # get dts
        dts,dts_offset, filtered_dts,filtered_dts_offset = TPQI.TPQI_analysis(folder_lt4,all_lt4,offsets,offsets_ch1,
                                               tpqi_dict['start_ch0'],tpqi_dict['start_ch0'],
                                               tpqi_dict['window_length'],
                                              tpqi_dict['pulse_sep'],
                                               tpqi_dict['number_of_pulses'],
                                               Verbose = False)
        ## calc visibility
        counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))
        counts_in_next_pulse = np.sum((filtered_dts_offset<200) & (filtered_dts_offset>-200))

        tail_lt3 = 7.2 #9.1975 for SIL 2 of pippin
        tail_lt4 = 5.1514

        p1 = tail_lt3
        p2 = tail_lt4

        Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
        if counts_in_same_pulse != 0 and counts_in_next_pulse != 0:
            vis.append(1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility)
            vis_u.append(np.sqrt((counts_in_same_pulse**2)/float(counts_in_next_pulse**3)+ \
                                 (counts_in_same_pulse)/float(counts_in_next_pulse**2))/Visibility)
            
#             print counts_in_same_pulse, counts_in_next_pulse,vis_u[-1]
        else:
            vis.append(0)
            vis_u.append(0)

        counts_filt.append(counts_in_same_pulse)
        counts_filt_next.append(counts_in_next_pulse)
    
    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.errorbar(sweep_pts,vis,yerr=vis_u)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Visibility')
    plt.show()
    plt.close('all')
    
    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.plot(sweep_pts,counts_filt,sweep_pts,counts_filt_next)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Counts')
    plt.show()
    plt.close('all')
    
    tpqi_dict[param_name] = initialVal
# Error propagation for N1/N2     Sqrt[N2^2 \[Sigma]1^2 + N1^2 \[Sigma]2^2]/N2^2

In [ ]:
##### sweep tpqi parameters
tpqi_dict = {
    'start_ch0' : 2772.5e3,
    'window_length' : 40e3,
    'pulse_sep' : 700e3,
    'number_of_pulses' : 1}

sweep_tpqi_param('start_ch0',np.linspace(2771e3,2781e3,10))
sweep_tpqi_param('window_length',np.linspace(10e3,60e3,10))

In [ ]:
def sweep_tpqi_days(steps):
    vis = []
    vis_u = []
    counts_filt = []
    counts_filt_next = []

    total_tstamps = len(all_lt4)
    file_ints = (np.linspace(0,total_tstamps,steps+1)).astype(int)

    for j in range(len(file_ints)-1):
        # get dts
        dts,dts_offset, filtered_dts,filtered_dts_offset = TPQI.TPQI_analysis(folder_lt4,all_lt4[file_ints[j]:file_ints[j+1]]
                                              ,offsets[file_ints[j]:file_ints[j+1]],offsets_ch1[file_ints[j]:file_ints[j+1]],tpqi_dict['start_ch0'],
                                              tpqi_dict['start_ch0'],
                                               tpqi_dict['window_length'],
                                              tpqi_dict['pulse_sep'],
                                               tpqi_dict['number_of_pulses'],
                                               Verbose = False)
        ## calc visibility
        counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))
        counts_in_next_pulse = np.sum((filtered_dts_offset<200) & (filtered_dts_offset>-200))

        tail_lt3 = 7.2 #9.1975 for SIL 2 of pippin
        tail_lt4 = 5.1514

        p1 = tail_lt3
        p2 = tail_lt4

        Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
        if counts_in_same_pulse != 0 and counts_in_next_pulse != 0:
            vis.append(1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility)
            vis_u.append(np.sqrt((counts_in_same_pulse**2)/float(counts_in_next_pulse**3)+ \
                                 (counts_in_same_pulse)/float(counts_in_next_pulse**2))/Visibility)
            
#             print counts_in_same_pulse, counts_in_next_pulse,vis_u[-1]
        else:
            vis.append(0)
            vis_u.append(0)

        counts_filt.append(counts_in_same_pulse)
        counts_filt_next.append(counts_in_next_pulse)
    
    sweep_pts = file_ints[1:]
    
    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.errorbar(sweep_pts,vis,yerr=vis_u)
    ax.set_xlabel('File nos')
    ax.set_ylabel('Visibility')
    plt.show()
    plt.close('all')

    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.plot(sweep_pts,counts_filt,sweep_pts,counts_filt_next)
    ax.set_xlabel('File nos')
    ax.set_ylabel('Counts')
    plt.show()
    plt.close('all')
    

In [ ]:
##### sweep tpqi parameters
tpqi_dict = {
    'start_ch0' : 2772.5e3,
    'window_length' : 40e3,
    'pulse_sep' : 700e3,
    'number_of_pulses' : 1}

sweep_tpqi_days(3)

In [ ]:
### regular histgram plotting:
reload(pq_plots)
# f = tb.latest_data('105117',folder = folder_lt4)
# a = pqsequence.PQSequenceAnalysis(f)
f = tb.latest_data(all_lt4[-1],folder = folder_lt4)
b = pqsequence.PQSequenceAnalysis(f)

# pq_plots.plot_photon_hist(a.pqf,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log = False)
pq_plots.plot_photon_hist(b.pqf,hist_binsize = 1e2,start=2.760e6,length=0.040e6,log = False)
# a.plot_histogram(0,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log_plot = False)
# a.plot_histogram(1,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log_plot = False)